In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

In [2]:
# CREDS = 'is3107-418808-5bca2b8d29d0.json'
CREDS = 'is3107-418011-f63573e5e1f3.json'
credentials = service_account.Credentials.from_service_account_file(CREDS)
client = bigquery.Client(credentials=credentials)
job_config = bigquery.QueryJobConfig()

In [3]:
dataset_id='is3107-418011.is3107'       
print(list(client.list_datasets(credentials.project_id)))
tables = client.list_tables(dataset_id)  
for table in tables:
    print(table.table_id)

Address
Property
Surroundings
Transaction
resale_price
testing
testing2


In [7]:
df = pd.read_csv('ETL/01_Data_Collection/01_dataset/processed_data/filtered_df3.csv')
df.head()

/var/folders/wx/4zj8vyds7kv6w4t2k82r4lcr0000gn/T/ipykernel_63345/621223118.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ETL/01_Data_Collection/01_dataset/processed_data/filtered_df3.csv')


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,school_type,bus_stop_proximity,mrt_proximity,pri_sch_proximity,sec_sch_proximity,latitude,longitude,inflation,normalized_resale_price,index
0,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0,...,elite,3-5 minutes,more than 15 minutes,3-5 minutes,3-5 minutes,1.374001,103.836432,1.322265,194372.998483,0
1,2000-02,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,130000.0,...,elite,3-5 minutes,more than 15 minutes,3-5 minutes,3-5 minutes,1.374001,103.836432,1.322265,171894.488454,1
2,2000-03,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,04 TO 06,45.0,Improved,1986,105000.0,...,elite,3-5 minutes,more than 15 minutes,3-5 minutes,3-5 minutes,1.374001,103.836432,1.322265,138837.856059,2
3,2000-05,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,04 TO 06,45.0,Improved,1986,125000.0,...,elite,3-5 minutes,more than 15 minutes,3-5 minutes,3-5 minutes,1.374001,103.836432,1.301036,162629.480554,3
4,2000-06,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,117000.0,...,elite,3-5 minutes,more than 15 minutes,3-5 minutes,3-5 minutes,1.374001,103.836432,1.301036,152221.193798,4


In [8]:
mixed_type_columns = df.select_dtypes(include='object').columns

if mixed_type_columns.empty:
    print("No columns with mixed data types.")
else:
    print("Columns with mixed data types:")
    print(mixed_type_columns)
    
df[mixed_type_columns] = df[mixed_type_columns].astype(str)

Columns with mixed data types:
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'flat_model', 'address', 'planning_area', 'mrt_name', 'bus_stop_name',
       'pri_sch_name', 'sec_sch_name', 'flat_type_model', 'school_type',
       'bus_stop_proximity', 'mrt_proximity', 'pri_sch_proximity',
       'sec_sch_proximity'],
      dtype='object')


In [ ]:
# months = ['January', 'February', 'March', 'April', 'May', 'June', 
#           'July', 'August', 'September', 'October', 'November', 'December']
# 
# df = pd.DataFrame({"month": months})
# df

In [26]:
property = df[['index', 'postal', 'flat_type', 'flat_model', 'flat_type_model',
                  'floor_area_sqm', 'lease_commence_date', 'storey_range',
                  'avg_storey_range', 'total_dwelling_units']]

# Drop duplicate rows based on the 'index' column
property = property.drop_duplicates(subset=['postal', 'flat_type_model',
                  'floor_area_sqm', 'lease_commence_date', 'storey_range',
                  'total_dwelling_units'])

property_duplicate_rows = property[property.duplicated()]

property_duplicate_rows


,index,postal,flat_type,flat_model,flat_type_model,floor_area_sqm,lease_commence_date,storey_range,avg_storey_range,total_dwelling_units


In [28]:
address = df[['postal', 'address', 'town', 'block', 'street_name', 
                          'planning_area', 'latitude', 'longitude']]

# Drop duplicate rows based on the 'index' column
address = address.drop_duplicates(subset=['postal'])

address_duplicate_rows = address[address.duplicated()]

address_duplicate_rows

,postal,address,town,block,street_name,planning_area,latitude,longitude


In [30]:
surroundings = df[['postal', 'commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion',
                               'total_dwelling_units', 'mall_nearest_distance', 'hawker_nearest_distance',
                               'hawker_food_stalls', 'hawker_market_stalls', 'mrt_nearest_distance', 'mrt_name',
                               'bus_interchange', 'mrt_interchange', 'bus_stop_nearest_distance', 'bus_stop_name',
                               'pri_sch_nearest_distance', 'pri_sch_name', 'vacancy', 'pri_sch_affiliation',
                               'sec_sch_nearest_dist', 'sec_sch_name', 'cutoff_point', 'affiliation', 'school_type',
                               'bus_stop_proximity', 'mrt_proximity', 'pri_sch_proximity', 'sec_sch_proximity']]

# Drop duplicate rows based on the 'index' column
surroundings = surroundings.drop_duplicates(subset=['postal'])

surroundings_duplicate_rows = surroundings[surroundings.duplicated()]

surroundings_duplicate_rows

,postal,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,mall_nearest_distance,hawker_nearest_distance,hawker_food_stalls,hawker_market_stalls,...,pri_sch_affiliation,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,school_type,bus_stop_proximity,mrt_proximity,pri_sch_proximity,sec_sch_proximity


In [36]:
transaction = df[['index', 'month', 'resale_price', 'year', 'quarter', 'inflation', 'normalized_resale_price']]

# Drop duplicate rows based on the 'index' column
transaction = transaction.drop_duplicates(subset=['month', 'resale_price', 'year', 'quarter', 'inflation', 'normalized_resale_price'])

transaction_duplicate_rows = transaction[transaction.duplicated()]

transaction_duplicate_rows


,index,month,resale_price,year,quarter,inflation,normalized_resale_price


In [12]:
project_id = "is3107-418011"

# Dataset ID and table name
dataset_id = "is3107"  # You have nested dataset, so specify only the innermost dataset
table_id = "resale_price"   # Table ID within the specified dataset

# Upload DataFrame to BigQuery
df.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8081%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=BMG7uWb8IPE0SCsarRwX5BNnzEvkvp&prompt=consent&access_type=offline


100%|██████████| 1/1 [00:00<00:00, 11397.57it/s]


In [27]:
# Dataset ID and table name
table_id = "Property"   # Table ID within the specified dataset

# Upload DataFrame to BigQuery
property.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 26214.40it/s]


In [29]:
# Dataset ID and table name
table_id = "Address"   # Table ID within the specified dataset

# Upload DataFrame to BigQuery
address.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 12985.46it/s]


In [31]:
# Dataset ID and table name
table_id = "Surroundings"   # Table ID within the specified dataset

# Upload DataFrame to BigQuery
surroundings.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 22192.08it/s]


In [37]:
# Dataset ID and table name
table_id = "Transaction"   # Table ID within the specified dataset

# Upload DataFrame to BigQuery
transaction.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 32263.88it/s]


In [5]:
def query_table_from_bq(table_name):
    CREDS = 'is3107-418011-f63573e5e1f3.json'
    credentials = service_account.Credentials.from_service_account_file(CREDS)
    client = bigquery.Client(credentials=credentials)
    job_config = bigquery.QueryJobConfig()

    # Set the destination table
    table = client.dataset(dataset_id='is3107-418011.is3107').table(table_id=f"is3107-418011.is3107.{table_name}")
    job_config.destination = table
    query = f"SELECT * FROM `is3107-418011.is3107.{table_name}`"
    return client.query(query).to_dataframe()

In [6]:
query_table_from_bq("testing2")

,month
0,January
1,February
2,March
3,April
4,May
5,June
6,July
7,August
8,September
9,October
